In [1]:
#CSV Bibliothek importieren 
import csv
#Pandas importieren und als pd bezeichnen
import pandas as pd
#Numpy importieren und als np bezeichnen 
import numpy as np
#Json Biliothek importieren
import json
#Obere Schranke für die Größe der Daten auf 100,000,000 setzen  
csv.field_size_limit(100000000)
#Maximale Anzahl von Zeilen und Spalten Zeigen 
pd.options.display.max_columns = None
pd.options.display.max_rows = None
#Extraktion der Domäne
import tldextract

***

# Moderate Polarisierung

1. Die Liste der polarisierten Domäne werden vorbereitet.
2. Die Anzahl der benutzten polarisierten Domäne in den Tweets wird berechnet
3. Die Polarisierung für die Benutzer wird berechnet
4. Zum Schluss werden die Ergebnisse gespeichert.

***

### Zuerst werden die Tweets in TweetsCov19 bearbeitet, dann die Tweets in TweetsKB
#### Vorbereitung der Domain-Labels

In [2]:
#Hier werden zuerst die Domain-Labels gelesen
pfad_tsv2 = 'C:/Mo/tweetDB/data/labels.csv'

df_label = pd.read_csv(pfad_tsv2)


In [3]:
#Die Quellen und Kategorien, die durch die Quelle unterstützt werden.
pd.options.display.max_columns = None
df_label.head()

,Unnamed: 0,"NewsGuard, Does not repeatedly publish false content","NewsGuard, Gathers and presents information responsibly","NewsGuard, Regularly corrects or clarifies errors","NewsGuard, Handles the difference between news and opinion responsibly","NewsGuard, Avoids deceptive headlines","NewsGuard, Website discloses ownership and financing","NewsGuard, Clearly labels advertising","NewsGuard, Reveals who's in charge, including any possible conflicts of interest","NewsGuard, Provides information about content creators","NewsGuard, score","NewsGuard, overall_class","Pew Research Center, known_by_40%","Pew Research Center, total","Pew Research Center, consistently_liberal","Pew Research Center, mostly_liberal","Pew Research Center, mixed","Pew Research Center, mostly conservative","Pew Research Center, consistently conservative","Wikipedia, is_fake","Open Sources, reliable","Open Sources, fake","Open Sources, unreliable","Open Sources, bias","Open Sources, conspiracy","Open Sources, hate","Open Sources, junksci","Open Sources, rumor","Open Sources, blog","Open Sources, clickbait","Open Sources, political","Open Sources, satire","Open Sources, state","Media Bias / Fact Check, label","Media Bias / Fact Check, factual_reporting","Media Bias / Fact Check, extreme_left","Media Bias / Fact Check, right","Media Bias / Fact Check, extreme_right","Media Bias / Fact Check, propaganda","Media Bias / Fact Check, fake_news","Media Bias / Fact Check, some_fake_news","Media Bias / Fact Check, failed_fact_checks","Media Bias / Fact Check, conspiracy","Media Bias / Fact Check, pseudoscience","Media Bias / Fact Check, hate_group","Media Bias / Fact Check, anti_islam","Media Bias / Fact Check, nationalism","Allsides, bias_rating","Allsides, community_agree","Allsides, community_disagree","Allsides, community_label","BuzzFeed, leaning","PolitiFact, Pants on Fire!","PolitiFact, False","PolitiFact, Mostly False","PolitiFact, Half-True","PolitiFact, Mostly True","PolitiFact, True"
0,21stCenturyWire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conspiracy_pseudoscience,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
1,ABC News,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,95.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_center_bias,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lean Left,8964.0,6949.0,somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AMERICAblog News,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
3,Activist Post,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conspiracy_pseudoscience,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
4,Addicting Info,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,left_bias,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Die Spalte 0 wird zu Domain umbenannt.
df_label.rename(columns = {'Unnamed: 0' : 'domain'}, inplace = True)
df_label['domain'] = df_label['domain'].str.replace(" ", "")
df_label['domain'] = df_label['domain'].str.lower()

In [5]:
pd.options.display.max_columns = None
df_label.head()

,domain,"NewsGuard, Does not repeatedly publish false content","NewsGuard, Gathers and presents information responsibly","NewsGuard, Regularly corrects or clarifies errors","NewsGuard, Handles the difference between news and opinion responsibly","NewsGuard, Avoids deceptive headlines","NewsGuard, Website discloses ownership and financing","NewsGuard, Clearly labels advertising","NewsGuard, Reveals who's in charge, including any possible conflicts of interest","NewsGuard, Provides information about content creators","NewsGuard, score","NewsGuard, overall_class","Pew Research Center, known_by_40%","Pew Research Center, total","Pew Research Center, consistently_liberal","Pew Research Center, mostly_liberal","Pew Research Center, mixed","Pew Research Center, mostly conservative","Pew Research Center, consistently conservative","Wikipedia, is_fake","Open Sources, reliable","Open Sources, fake","Open Sources, unreliable","Open Sources, bias","Open Sources, conspiracy","Open Sources, hate","Open Sources, junksci","Open Sources, rumor","Open Sources, blog","Open Sources, clickbait","Open Sources, political","Open Sources, satire","Open Sources, state","Media Bias / Fact Check, label","Media Bias / Fact Check, factual_reporting","Media Bias / Fact Check, extreme_left","Media Bias / Fact Check, right","Media Bias / Fact Check, extreme_right","Media Bias / Fact Check, propaganda","Media Bias / Fact Check, fake_news","Media Bias / Fact Check, some_fake_news","Media Bias / Fact Check, failed_fact_checks","Media Bias / Fact Check, conspiracy","Media Bias / Fact Check, pseudoscience","Media Bias / Fact Check, hate_group","Media Bias / Fact Check, anti_islam","Media Bias / Fact Check, nationalism","Allsides, bias_rating","Allsides, community_agree","Allsides, community_disagree","Allsides, community_label","BuzzFeed, leaning","PolitiFact, Pants on Fire!","PolitiFact, False","PolitiFact, Mostly False","PolitiFact, Half-True","PolitiFact, Mostly True","PolitiFact, True"
0,21stcenturywire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conspiracy_pseudoscience,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
1,abcnews,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,95.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_center_bias,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lean Left,8964.0,6949.0,somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,americablognews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
3,activistpost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conspiracy_pseudoscience,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
4,addictinginfo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,left_bias,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#Die beinhalteten Values für Media Bias / Fact Check zählen 
df_label['Allsides, bias_rating'].value_counts()

Left          17
Right         14
Lean Left     13
Center        11
Lean Right    10
Name: Allsides, bias_rating, dtype: int64

In [8]:
#Die beinhalteten Values für Media Bias / Fact Check, extreme_left zählen 
df_label['Media Bias / Fact Check, extreme_left'].value_counts()

0.0    133
1.0      2
Name: Media Bias / Fact Check, extreme_left, dtype: int64

In [9]:
#Die beinhalteten Values für Media Bias / Fact Check, extreme_right zählen 
df_label['Media Bias / Fact Check, extreme_right'].value_counts()

0.0    123
1.0     12
Name: Media Bias / Fact Check, extreme_right, dtype: int64

In [10]:
#Die beinhalteten Values für Media Bias / Fact Check, right zählen 
df_label['Media Bias / Fact Check, right'].value_counts()

0.0    134
1.0      1
Name: Media Bias / Fact Check, right, dtype: int64

In [11]:
#Die beinhalteten Values für Media Bias / Fact Check, label zählen 
df_label['Media Bias / Fact Check, label'].value_counts()

left_center_bias            36
left_bias                   30
conspiracy_pseudoscience    17
right_bias                  16
questionable_source         15
right_center_bias           10
satire                       8
least_biased                 3
Name: Media Bias / Fact Check, label, dtype: int64

- Moderate Domäne:

In [12]:
#Die Labels von Allsides, bias_rating, die mit Lean Left markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Allsides, bias_rating'] == 'Lean Left']

In [13]:
list_mod = []
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_mod.extend(l_Media_Bias)

In [14]:
#Die Labels von Allsides, bias_rating, die mit Lean Right markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Allsides, bias_rating'] == 'Lean Right']

In [15]:
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_mod.extend(l_Media_Bias)

In [16]:
#Die Labels von Media Bias / Fact Check, extreme_left, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Media Bias / Fact Check, left'] == 1.0]

In [17]:
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_mod.extend(l_Media_Bias)

In [18]:
#Die Labels von Media Bias / Fact Check, label, die mit left_center_bias markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Media Bias / Fact Check, label'] == 'left_center_bias']

In [19]:
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_mod.extend(l_Media_Bias)

In [20]:
#Die Labels von Media Bias / Fact Check, label, die mit right_center_bias markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Media Bias / Fact Check, label'] == 'right_center_bias']

In [21]:
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_mod.extend(l_Media_Bias)

In [32]:
list_mod.append('californiaglobe')

In [33]:
list_mod = np.array(list_mod)
list_mod = np.unique(list_mod)
list_mod = list_mod.tolist()

In [34]:
#Anzahl der Domäne
len(list_mod)

187

In [10]:
#Liste der moderaten Domäne
list_mod = np.array(list_mod)
list_mod.reshape(1,-1)

array([['19fortyfive', '604now', '7news', '972mag', '9news',
        'Deutschland', 'abc', 'abc11', 'abcnews', 'abovethelaw',
        'abqjournal', 'aceshowbiz', 'aclu', 'acsh', 'adl', 'aeon',
        'affinitymagazine', 'africanarguments', 'africanews', 'ahram',
        'ahvalnews', 'aier', 'al', 'al-monitor', 'al-sura',
        'albertaviews', 'aljazeera', 'allgov', 'alliedprogress',
        'americamagazine', 'americanactionforum',
        'americanaffairsjournal', 'americanfirearms',
        'americanimmigrationcouncil', 'americanpublicmedia',
        'americanpurpose', 'americasquarterly', 'amnesty', 'amny',
        'aninews', 'annistonstar', 'anthropocenemagazine', 'antiwar',
        'aol', 'apmreports', 'appledaily', 'arcamax', 'arkansasonline',
        'arktimes', 'armscontrol', 'arte', 'atlanticcouncil', 'aynrand',
        'azcentral', 'azdailysun', 'bbc', 'beaconjournal',
        'beinglibertarian', 'berlingske', 'bild', 'billingsgazette',
        'bipartisanpolicy', 'birming

In [26]:
#Die Moderaten speichern
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/dom_moderat_2.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for i  in range (0, len(list_mod)):
        l = []
        l.append(list_mod[i])
        tsv_writer.writerow(l)

***

- Hardliner-Domäne:

In [117]:
#Die Labels von Allsides, bias_rating', die mit Right markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Allsides, bias_rating'] == 'Right']

In [119]:
list_hard = []
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_hard.extend(l_Media_Bias)

In [121]:
#Die Labels von Allsides, bias_rating', die mit Left markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Allsides, bias_rating'] == 'Left']

In [122]:
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_hard.extend(l_Media_Bias)

In [124]:
#Die Labels von Media Bias / Fact Check, label, die mit left_bias markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Media Bias / Fact Check, label'] == 'left_bias']

In [125]:
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_hard.extend(l_Media_Bias)

In [126]:
#Die Labels von Media Bias / Fact Check, right, die mit right_bias markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Media Bias / Fact Check, label'] == 'right_bias']

In [127]:
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_hard.extend(l_Media_Bias)

In [19]:
list_hard.append('')

In [20]:
list_hard = np.array(list_hard)
list_hard = np.unique(list_hard)
list_hard = list_hard.tolist()

In [11]:
#Liste der Hirdliner-Domäne 
list_hard = np.array(list_hard)
list_hard.reshape(1,-1)

array([['2ndvote', 'Aish', 'AsianNews', 'CAGW', 'DonaldJTrump', 'aa',
        'ac2news', 'academia', 'achnews', 'aclj', 'act', 'addictinginfo',
        'aei', 'afj', 'afpc', 'aidc', 'aim', 'aina', 'alec', 'alternet',
        'altnewsmedia', 'amac', 'americablog', 'americanbridgepac',
        'americanconsequences', 'americanexperiment',
        'americanlibertyreport', 'americannewsx', 'americanprogress',
        'americansforprosperity', 'americarisingpac', 'amgreatness',
        'amimagazine', 'ammoland', 'angrywhitemen', 'antifascistnews',
        'arizonadailyindependent', 'armstrongeconomics',
        'attackthesystem', 'au', 'autostraddle', 'axiomnewsnetwork',
        'bearingarms', 'beliefnet', 'billmoyers', 'bitchmedia',
        'bizpacreview', 'blabber', 'blackagendareport',
        'blackamericaweb', 'blacklivesmatter', 'blackmainstreet',
        'bluelivesmatter', 'bluestatedaily', 'boingboing',
        'bostonreview', 'boundingintocomics', 'breitbart',
        'brexitcentra

In [38]:
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/dom_hardliner.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for i  in range (0, len(list_hard)):
        l = []
        l.append(list_hard[i])
        tsv_writer.writerow(l)

***

## Polarisierungen berechnen
### TweetsCov19

***

In [2]:
#Die liste der anti-wissenschaftliche Domäne lesen.
pfad = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/dom_moderat_2.tsv'

df_dom_mod = pd.read_csv(pfad, sep='\t', error_bad_lines=False, names=['dom'])

In [3]:
#Die Spalte der moderaten Domöne zu Liste umwandeln.
list_mod = df_dom_mod.dom.tolist()

In [4]:
list_mod = np.array(list_mod)
list_mod = np.unique(list_mod)
list_mod = list_mod.tolist()

In [5]:
len(list_mod)

187

In [6]:
#Die liste der pro-wissenschaftliche Domäne lesen.
pfad = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/dom_hardliner.tsv'

df_dom_hard = pd.read_csv(pfad, sep='\t', error_bad_lines=False, names=['dom'])

In [7]:
#Die Spalte der Hardliner-Domäne zu Liste umwandeln.
list_hard =df_dom_hard.dom.tolist()

In [8]:
list_hard = np.array(list_hard)
list_hard = np.unique(list_hard)
list_hard = list_hard.tolist()

In [9]:
len(list_hard)

187

In [72]:
#Alle tweets als Datframe lesen.
pfad_tsv1 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsCov19_sample/url/tweetsCov19_id_is_dom.tsv'

df_tweetscov19 = pd.read_csv(pfad_tsv1, sep='\t', error_bad_lines=False, names=['Tweet_Id', 'Username','is_deleted','Urls_num', 'Url_dom'])
pd.options.display.max_columns = None
print('Shape :', df_tweetscov19.shape)
df_tweetscov19.head()

Shape : (9409841, 5)


,Tweet_Id,Username,is_deleted,Urls_num,Url_dom
0,1178791637348212736,c57d4fb895c5f4a33d2a2b24709f93b5,True,0,0
1,1178791641601232896,627f7d4ef584ba33a676a4ebcb861cdb,False,1,catholicnewsagency.com
2,1178791643509612547,7966d5716c1c4f33591b17755184e3d6,False,0,0
3,1178791644214235137,f17996c036ed4cd981f5448fe00d9ed2,False,0,0
4,1178791645371875329,32a54e5f6d8a67de1bffd2e689594dc8,False,0,0


In [73]:
#Diese Methode bekommt benutzten Domäne als ein String,
#Wandelt sie die String-daten zu Liste um.
#Die Methode geht die Liste durch, prüft jede Domain. 
#Wenn Domain in moderaten Domäne wäre, dann bekommt der Tweet eine Wertung von -1.


def get_mod_pol(domains):
    domains_list = domains.split()
    score = 0
    for i in range(0, len(domains_list)):
        
        d = tldextract.extract(domains_list[i])
        d = d.domain
                   
        
        if d in list_mod:
            score -=  1
        
    return score

In [74]:
#Für alle Tweets wird die Anzahl der benutzten moderaten Domäne berechnet.
df_tweetscov19['Url_dom'] = df_tweetscov19['Url_dom'].astype(str)
df_tweetscov19['dom_mod_pol'] = df_tweetscov19.apply(lambda row: get_mod_pol(row['Url_dom']) if (row['Url_dom'] != '0') else 0, axis=1)

In [75]:
df_tweetscov19.head()

,Tweet_Id,Username,is_deleted,Urls_num,Url_dom,dom_mod_pol
0,1178791637348212736,c57d4fb895c5f4a33d2a2b24709f93b5,True,0,0,0
1,1178791641601232896,627f7d4ef584ba33a676a4ebcb861cdb,False,1,catholicnewsagency.com,0
2,1178791643509612547,7966d5716c1c4f33591b17755184e3d6,False,0,0,0
3,1178791644214235137,f17996c036ed4cd981f5448fe00d9ed2,False,0,0,0
4,1178791645371875329,32a54e5f6d8a67de1bffd2e689594dc8,False,0,0,0


In [76]:
df_tweetscov19[df_tweetscov19['dom_mod_pol'] < 0].shape

(246600, 6)

In [77]:
#Diese Methode bekommt benutzten Domäne als ein String,
#Wandel sie die String-daten zu Liste um.
#Die Methode geht die Liste durch, prüft jede Domain. 
#Wenn Domain in Hardliner-Domäne wäre, dann bekommt der Tweet eine Wertung von 1

def get_hard_pol(domains):
    domains_list = domains.split()
    score = 0
    for i in range(0, len(domains_list)):
        
        d = tldextract.extract(domains_list[i])
        d = d.domain
       
            
        
        if d in list_hard:
            score +=  1
        
    return score

In [78]:
#Für alle Tweets wird die anzahl der benutzten konservativen Domäne berechnet.
df_tweetscov19['dom_hard_pol'] = df_tweetscov19.apply(lambda row: get_hard_pol(row['Url_dom']) if (row['Url_dom'] != '0') else 0, axis=1)

In [79]:
df_tweetscov19.head()

,Tweet_Id,Username,is_deleted,Urls_num,Url_dom,dom_mod_pol,dom_hard_pol
0,1178791637348212736,c57d4fb895c5f4a33d2a2b24709f93b5,True,0,0,0,0
1,1178791641601232896,627f7d4ef584ba33a676a4ebcb861cdb,False,1,catholicnewsagency.com,0,0
2,1178791643509612547,7966d5716c1c4f33591b17755184e3d6,False,0,0,0,0
3,1178791644214235137,f17996c036ed4cd981f5448fe00d9ed2,False,0,0,0,0
4,1178791645371875329,32a54e5f6d8a67de1bffd2e689594dc8,False,0,0,0,0


In [80]:
df_tweetscov19[df_tweetscov19['dom_hard_pol'] > 0].shape

(145972, 7)

In [81]:
#Hier werden die Benutzer in Dataframe gruppiert, und die Features 'Anzahl der Domäne', 'Negative Wertungen' und 'Psoitive Wertungen summiert.' 
#dom_mod_pol = Moderaten
#dom_hard_pol = Hardliner
df_tweetscov19_unique_user = df_tweetscov19.groupby('Username')['Urls_num', 'dom_mod_pol', 'dom_hard_pol'].sum()

<ipython-input-81-57eecac3af36>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_tweetscov19_unique_user = df_tweetscov19.groupby('Username')['Urls_num', 'dom_mod_pol', 'dom_hard_pol'].sum()


In [82]:
df_tweetscov19_unique_user.shape

(4095876, 3)

In [83]:
df_tweetscov19_unique_user[df_tweetscov19_unique_user['dom_hard_pol']>0].head()

,Urls_num,dom_mod_pol,dom_hard_pol
Username,,,
00012a177385cd6e7590e2acb3f206e3,1,-1,1
0001a863c159122c712c53258cf13c8b,1,0,1
000353a523eec0084e195fafc082a6a9,1,0,1
0003e8427d9272dd24f2bcb66b871ea9,2,0,2
0004878b3fd069f70a0277acafeffad1,1,0,1


In [84]:
#Anzahl der benutzten polarisierten Domäne wird berechnet.
df_tweetscov19_unique_user['pol_num'] = df_tweetscov19_unique_user.apply(lambda row: (row['dom_hard_pol'] - row['dom_mod_pol']), axis=1)

In [85]:
df_tweetscov19_unique_user[df_tweetscov19_unique_user['dom_mod_pol']<0].head()

,Urls_num,dom_mod_pol,dom_hard_pol,pol_num
Username,,,,
00008ca06f863b962f72cc9167d7978a,1,-1,0,1
00012a177385cd6e7590e2acb3f206e3,1,-1,1,2
00016e20a3865cb74581803c27d27b89,1,-1,0,1
000220d9d43b6b1ad7dbbdbfbcc53371,5,-2,0,2
00037260d5e4fe01965a48a96974b53f,2,-2,0,2


In [86]:
#Die Polarisierung der Domäne wird berechnet.
#Anzahl der Moderatne + der Hardliner durch die Anzahl der benutzten Domäne.
df_tweetscov19_unique_user['user_pol'] = df_tweetscov19_unique_user.apply(lambda row: (row['dom_mod_pol'] + row['dom_hard_pol']) / row['pol_num'] if row['pol_num'] != 0 else 0, axis=1)

In [87]:
df_tweetscov19_unique_user[df_tweetscov19_unique_user['dom_mod_pol']<-2].head()

,Urls_num,dom_mod_pol,dom_hard_pol,pol_num,user_pol
Username,,,,,
00065a00b971b5e1629d504e0ddf2da0,27,-6,2,8,-0.500000
0013aaa4da0461c58d0b1306af75af02,215,-16,23,39,0.179487
00247be59d748ab0e377ab5885ecc66b,7,-5,4,9,-0.111111
0028b1c0f33fc484e49c2ce2820cd4ae,25,-4,0,4,-1.000000
002f506ad2fa8c2cb570fa65d135de67,24,-8,12,20,0.200000


In [88]:
#Die Benutzer, die eine Wertung von 0 bekommen haben, werden ausgenommen.
df_tweetscov19_unique_user_0 = df_tweetscov19_unique_user[df_tweetscov19_unique_user['user_pol'] != 0]

In [89]:
#Die Form der Dataframe
df_tweetscov19_unique_user_0.shape

(96513, 5)

In [90]:
#Die Form der Dataframe Hardliner.
df_tweetscov19_unique_user_0[df_tweetscov19_unique_user_0['user_pol'] > 0].shape

(24804, 5)

In [91]:
#Die Form der Dataframe Moderaten.
df_tweetscov19_unique_user_0[df_tweetscov19_unique_user_0['user_pol'] < 0].shape

(71709, 5)

In [97]:
#Die polarisieten Werte für Tweets werden gespeichert.
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/tweetscov19_dom_moderacy.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetscov19.iterrows():
        tsv_writer.writerow(row.values.tolist())

In [ ]:
df_tweetscov19_unique_user_0 = df_tweetscov19_unique_user_0.reset_index()

In [ ]:
#Die Benutzer mit bekannten polarisierten Ansichten speichern. 
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/tweetscov19_dom_moderacy.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetscov19_unique_user_0.iterrows():
        tsv_writer.writerow(row.values.tolist())

***

## Polarisierungen berechnen
### TweetsKB

***

In [92]:
#Dataframe wird gelesen.
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/tweetskb_sample/url/tweetskb_sample_id_is_urls.tsv'

df_tweetskb = pd.read_csv(pfad_tsv3, sep='\t', error_bad_lines=False, names=['Tweet_Id', "Username", 'is_deleted', 'URLs_dom', 'URLs_num'])
pd.options.display.max_columns = None
print('Shape :', df_tweetskb.shape)
df_tweetskb.head()

Shape : (9409841, 5)


,Tweet_Id,Username,is_deleted,URLs_dom,URLs_num
0,1178791636039589889,DCComics,False,dcuniverseinfinite.com,1
1,1178791641265639424,seanberdyz,True,carrd.co,1
2,1178791640573579264,AnArtistAtBirth,True,0,0
3,1178791644797255680,lagosboygang,True,0,0
4,1178791648056217600,Phenom_Hoops,False,phenomhoopreport.com,1


In [93]:
#Für alle Domäne in Tweets wird die Anzahl der moderaten Domäne berechnet.
df_tweetskb['URLs_dom'] = df_tweetskb['URLs_dom'].astype(str)
df_tweetskb['dom_mod_pol'] = df_tweetskb.apply(lambda row: get_mod_pol(row['URLs_dom']) if (row['URLs_dom'] != '0') else 0, axis=1)

In [94]:
df_tweetskb[df_tweetskb['dom_mod_pol'] < 0].shape

(53430, 6)

In [61]:
#Für alle Domäne in Tweets wird die Anzahl der Hardliner-Domäne berechnet.
df_tweetskb['dom_hard_pol'] = df_tweetskb.apply(lambda row: get_hard_pol(row['URLs_dom']) if (row['URLs_dom'] != '0') else 0, axis=1)

In [62]:
df_tweetskb[df_tweetskb['dom_hard_pol'] > 0].shape

(35518, 7)

In [63]:
#Hier werden die Benutzer in Dataframe gruppiert, und die Features 'Anzahl der Domäne', 'Negative Wertungen' und 'Psoitive Wertungen summiert.' 
#dom_mod_pol = Liberalen
#dom_hard_pol = Konservativen
df_tweetskb_unique_user = df_tweetskb.groupby('Username')['URLs_num', 'dom_mod_pol', 'dom_hard_pol'].sum()

<ipython-input-63-5ed131bb2c86>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_tweetskb_unique_user = df_tweetskb.groupby('Username')['URLs_num', 'dom_mod_pol', 'dom_hard_pol'].sum()


In [64]:
df_tweetskb_unique_user.shape

(5208425, 3)

In [65]:
df_tweetskb_unique_user.head()

,URLs_num,dom_mod_pol,dom_hard_pol
Username,,,
00000000,0,0,0
000000000yt,0,0,0
00000000martin,0,0,0
000000911a,0,0,0
000000_127,0,0,0


In [66]:
#Die Anzahl der benutzten polarisierten Domäne werden berechnet.
df_tweetskb_unique_user['pol_num'] = df_tweetskb_unique_user.apply(lambda row:  (row['dom_hard_pol'] - row['dom_mod_pol']), axis=1)

In [67]:
df_tweetskb_unique_user[df_tweetskb_unique_user['dom_mod_pol']<-2].head()

,URLs_num,dom_mod_pol,dom_hard_pol,pol_num
Username,,,,
10F01C0,7,-3,1,4
1120Eclipse,20,-7,4,11
13SciDave,4,-3,0,3
13thethe,8,-3,3,6
1944jnewell,3,-3,0,3


In [68]:
#Die Polarisierungswerte für die Benutzer wird berechnet.
df_tweetskb_unique_user['user_pol'] = df_tweetskb_unique_user.apply(lambda row: (row['dom_mod_pol'] + row['dom_hard_pol']) / row['pol_num'] if row['pol_num'] != 0 else 0, axis=1)

In [69]:
#Die Benutzer, die eine Wertung von 0 bekommen haben, werden ausgenommen.
df_tweetskb_unique_user_0 = df_tweetskb_unique_user[df_tweetskb_unique_user['user_pol'] != 0]

In [70]:
#Die Form der Dataframe
df_tweetskb_unique_user_0.shape

(34352, 5)

In [71]:
#Die Hardliner 
df_tweetskb_unique_user_0[df_tweetskb_unique_user_0['user_pol'] > 0].shape

(9040, 5)

In [72]:
#Die Moderaten
df_tweetskb_unique_user_0[df_tweetskb_unique_user_0['user_pol'] < 0].shape

(25312, 5)

In [129]:
#Die polarisierten Werte für die Tweets werden gespeichert.
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/tweetskb_dom_moderacy.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetskb.iterrows():
        tsv_writer.writerow(row.values.tolist())

In [ ]:
df_tweetskb_unique_user_0 = df_tweetskb_unique_user_0.reset_index()

In [ ]:
#Die Benutzer mit polarisierten Werte werden gespeichert.
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/tweetskb_dom_moderacy.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetskb_unique_user_0.iterrows():
        tsv_writer.writerow(row.values.tolist())